<p style="font-size:32px;text-align:center"> <b>Social network Graph Link Prediction - Facebook Challenge</b> </p>

In [1]:
#Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os

import warnings
import networkx as nx

import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm

In [2]:
import modin.pandas as modpd

In [3]:
os.environ["MODIN_ENGINE"] = "ray"

# 1. Reading Data

In [4]:
if os.path.isfile('saved_after_eda/train_pos_after_eda.csv'):
    train_graph=nx.read_edgelist('saved_after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
    print(nx.info(train_graph))
else:
    print("please run the FB_EDA.ipynb or download the files from drive")

Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


In [5]:
#1.7M nodes and 7.5M edges in train pos data, we derive all our features from this

# 2. Similarity measures
## Jaccard Distance, Cosine Similarity, Preferential Attachment 

Jaccard, Cosine and Preferential make use of common terms like predecessors set, successors set and length of these sets so we club them together under one function, compute the required common terms only once and make repeated use of these to compute the features.

In [6]:
def featurizationSet1(a,b):
    #calculate all the terms needed to derive the below features only once to save time
    try:
        predecessors_a = set(train_graph.predecessors(a))
        predecessors_b = set(train_graph.predecessors(b))
    
        len_pred_a = len(predecessors_a)
        len_pred_b = len(predecessors_b)
    except:
        len_pred_a = 0
        len_pred_b = 0
        
    try:
        successors_a = set(train_graph.successors(a))
        successors_b = set(train_graph.successors(b))
        
        len_succ_a = len(successors_a)
        len_succ_b = len(successors_b)
    except:
        len_succ_a = 0
        len_succ_b = 0
    
    results = []
    
    #x----------------------------------------------------------------------------------------------------------------------x
    #derive all the features from the terms computed above
    
    #jaccard_for_followees
    try:
        if len_succ_a == 0  | len_succ_b == 0:
            results.append(0)
        else:
            sim = (len(successors_a.intersection(successors_b)))/(len(successors_a.union(successors_b)))
            results.append(sim)
    except:
        results.append(0)
    
    #jaccard_for_followers
    try:
        if len_pred_a == 0  | len_pred_b == 0: 
            results.append(0)
        else:
            sim = (len(predecessors_a.intersection(predecessors_b)))/(len(predecessors_a.union(predecessors_b)))
            results.append(sim)
    except:
        results.append(0)
    
    #cosine_for_followees
    try:
        if len_succ_a == 0  | len_succ_b == 0:
            results.append(0)
        else:
            sim = (len(successors_a.intersection(successors_b)))/(math.sqrt(len_succ_a*len_succ_b))
            results.append(sim)
    except:
        results.append(0)
    
    #cosine_for_followers
    try:
        if len_pred_a == 0  | len_pred_b == 0:
            results.append(0)
        else:
            sim = (len(predecessors_a.intersection(predecessors_b)))/(math.sqrt(len_pred_a*len_pred_b))
            results.append(sim)
    except:
        results.append(0)
    
    #preferential_for_followees
    try:
        if len_succ_a == 0  | len_succ_b == 0:
            results.append(0)
        else:
            score = len_succ_a * len_succ_b
            results.append(score)
    except:
        results.append(0)

    #preferential_for_followers(a,b)
    try:
        if len_pred_a == 0  | len_pred_b == 0:
            results.append(0)
        else:
            score = len_pred_a * len_pred_b
            results.append(score)
    except:
        results.append(0)
    
    return results

In [7]:
%%time
featurizationSet1(4345,1502)

CPU times: user 27 µs, sys: 4 µs, total: 31 µs
Wall time: 35.8 µs


[0.0, 0.0, 0.0, 0, 2, 0]

# 3. Ranking Measures
## Page Ranking
https://en.wikipedia.org/wiki/PageRank

In [8]:
if not os.path.isfile('saved_fea/page_rank.p'):
    pr = nx.pagerank(train_graph, alpha=0.9)
    pickle.dump(pr, open('saved_fea/page_rank.p', 'wb'))
else:
    pr = pickle.load(open('saved_fea/page_rank.p','rb'))

# 4. Other Graph Features
## Shortest path

Getting Shortest path between two nodes, if nodes have direct path i.e directly connected then we are removing that edge and calculating path. 

In [9]:
#if has direct edge then deleting that edge and calculating shortest path
def compute_shortest_path_length(a,b):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1

In [10]:
#testing
print(compute_shortest_path_length(77697, 826021))
print(compute_shortest_path_length(669354,1635354))

10
-1


## Checking for same community

In [11]:
#getting weekly connected edges from graph 
wcc=list(nx.weakly_connected_components(train_graph))
def belongs_to_same_wcc(a,b):
    index = []
    if train_graph.has_edge(b,a):
        return 1
    if train_graph.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                train_graph.remove_edge(a,b)
                if compute_shortest_path_length(a,b)==-1:
                    train_graph.add_edge(a,b)
                    return 0
                else:
                    train_graph.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

In [12]:
print('Total weakly conncected components in train graph:',len(wcc))

Total weakly conncected components in train graph: 48602


In [13]:
#testing
print(belongs_to_same_wcc(669354,1635354))
print(belongs_to_same_wcc(861, 1659750))

0
0


## Adamic/Adar Index
Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

In [14]:
#adar index
def calc_adar_in(a,b):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

In [15]:
#testing
print(calc_adar_in(1,189226))
print(calc_adar_in(669354,1635354))

0
0


## Follow Back (Is persion was following back)

In [16]:
def follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

In [17]:
#testing
print(follows_back(1,189226))
print(follows_back(669354,1635354))

1
0


## Katz Centrality
https://en.wikipedia.org/wiki/Katz_centrality

https://www.geeksforgeeks.org/katz-centrality-centrality-measure/
 Katz centrality computes the centrality for a node 
    based on the centrality of its neighbors. It is a 
    generalization of the eigenvector centrality. The
    Katz centrality for node `i` is
 
$$x_i = \alpha \sum_{j} A_{ij} x_j + \beta,$$
where `A` is the adjacency matrix of the graph G 
with eigenvalues $$\lambda$$.

The parameter $$\beta$$ controls the initial centrality and 

$$\alpha < \frac{1}{\lambda_{max}}.$$

In [18]:
if not os.path.isfile('saved_fea/katz.p'):
    katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
    pickle.dump(katz,open('saved_fea/katz.p','wb'))
else:
    katz = pickle.load(open('saved_fea/katz.p','rb'))

In [19]:
print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
print('mean',float(sum(katz.values())) / len(katz))

min 0.0007313532484065916
max 0.003394554981699122
mean 0.0007483800935562018


## Hits Score
The HITS algorithm computes two numbers for a node. Authorities estimates the node value based on the incoming links. Hubs estimates the node value based on outgoing links.

https://en.wikipedia.org/wiki/HITS_algorithm

In [20]:
if not os.path.isfile('saved_fea/hits.p'):
    hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    pickle.dump(hits,open('saved_fea/hits.p','wb'))
else:
    hits = pickle.load(open('saved_fea/hits.p','rb'))

In [21]:
print('min',hits[0][min(hits[0], key=hits[0].get)])
print('max',hits[0][max(hits[0], key=hits[0].get)])
print('mean',float(sum(hits[0].values())) / len(hits[0]))

min 0.0
max 0.004868653378780953
mean 5.615699699344123e-07


# 5. Featurization

- Use modpd in place of pd for all operations.
- We make use of the complete data here

In [22]:
%%time
df_train = modpd.read_csv('saved_after_eda/train_after_eda.csv', names=['source_node', 'destination_node'])
df_train['indicator_link'] = modpd.read_csv('saved_after_eda/train_y.csv', names=['indicator_link'])
print(df_train.shape)
df_train.head()

(15100030, 3)
CPU times: user 939 ms, sys: 1.08 s, total: 2.02 s
Wall time: 7.47 s


,source_node,destination_node,indicator_link
0,273084,1505602,1
1,912810,1678443,1
2,365429,1523458,1
3,527014,1605979,1
4,1228116,471233,1


In [23]:
%%time
df_test =  modpd.read_csv('saved_after_eda/test_after_eda.csv', names=['source_node', 'destination_node'])
df_test['indicator_link'] = modpd.read_csv('saved_after_eda/test_y.csv', names=['indicator_link'])
print(df_test.shape)
df_test.head()

(3775008, 3)
CPU times: user 195 ms, sys: 122 ms, total: 317 ms
Wall time: 658 ms


,source_node,destination_node,indicator_link
0,848424,784690,1
1,1248963,444518,1
2,264224,132395,1
3,549680,326829,1
4,875380,1394902,1


In [24]:
#use copy of train and test for creating new features
df_train_cpy = df_train.copy()
df_test_cpy = df_test.copy()

## 5.1 Adding a set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>jaccard_followers</li>
<li>jaccard_followees</li>
<li>cosine_followers</li>
<li>cosine_followees</li>
<li>preferential_for_followers</li>
<li>preferential_for_followees</li>  
<li>num_followers_s</li>
<li>num_followees_s</li>
<li>num_followers_d</li>
<li>num_followees_d</li>
<li>inter_followers</li>
<li>inter_followees</li>
</ol>

#### Adding : num_followers, num_followees, inter_followers, inter_followees for source and destination nodes

In [25]:
def compute_feat(src, dest):
    try:
        s1=set(train_graph.predecessors(src))
        s2=set(train_graph.successors(src))
    except:
        s1 = set()
        s2 = set()
    try:
        d1=set(train_graph.predecessors(dest))
        d2=set(train_graph.successors(dest))
    except:
        d1 = set()
        d2 = set()
    
    src_p = len(s1)
    src_s = len(s2)
    
    dest_p = len(d1)
    dest_s = len(d2)
    
    inter_p = len(s1.intersection(d1))
    inter_s = len(s2.intersection(d2))
    
    return  src_p, dest_p, src_s, dest_s, inter_p, inter_s

In [26]:
#converting to array and then operating improves the speed 

In [27]:
%%time
if not os.path.isfile('saved_after_fea/storage_stage1.h5'):
    arr_train_cpy = np.array(df_train_cpy)
    arr_test_cpy = np.array(df_test_cpy)
    
    new_feat_train_cpy = np.array(list(map(lambda x: compute_feat(x[0], x[1]), arr_train_cpy)))
    stacked_train = np.hstack((arr_train_cpy, new_feat_train_cpy))
    df_train_cpy = modpd.DataFrame(stacked_train, columns = ['source_node','destination_node','indicator_link', 'num_followers_s',
                                                          'num_followers_d','num_followees_s', 'num_followees_d', 
                                                          'inter_followers', 'inter_followees'])
    
    new_feat_test_cpy = np.array(list(map(lambda x: compute_feat(x[0], x[1]), arr_test_cpy)))
    stacked_test = np.hstack((arr_test_cpy, new_feat_test_cpy))
    df_test_cpy = modpd.DataFrame(stacked_test, columns = ['source_node','destination_node','indicator_link', 'num_followers_s',
                                                        'num_followers_d','num_followees_s', 'num_followees_d', 
                                                        'inter_followers', 'inter_followees'])
    
    hdf = modpd.HDFStore('saved_after_fea/storage_stage1.h5')
    hdf.put('train_df',df_train_cpy, format='table', data_columns=True)
    hdf.put('test_df',df_test_cpy, format='table', data_columns=True)
    hdf.close()
else:
    df_train_cpy = modpd.read_hdf('saved_after_fea/storage_stage1.h5', 'train_df',mode='r')
    df_test_cpy = modpd.read_hdf('saved_after_fea/storage_stage1.h5', 'test_df',mode='r')

CPU times: user 8.1 s, sys: 14.1 s, total: 22.2 s
Wall time: 21.7 s


In [28]:
#checking results
print(df_train_cpy.shape)
df_train_cpy.head()

(15100030, 9)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees
0,273084,1505602,1,11,6,15,8,0,0
1,912810,1678443,1,10,8,10,8,1,1
2,365429,1523458,1,40,85,49,84,4,3
3,527014,1605979,1,0,1,1,0,0,0
4,1228116,471233,1,14,48,23,20,4,6


In [29]:
print(df_test_cpy.shape)
df_test_cpy.head()

(3775008, 9)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees
0,848424,784690,1,6,14,6,9,1,0
1,1248963,444518,1,5,1,8,2,0,0
2,264224,132395,1,8,3,7,7,3,4
3,549680,326829,1,17,12,11,15,3,1
4,875380,1394902,1,21,29,20,25,8,7


#### Adding: jaccard_followers, jaccard_followees, cosine_followers, cosine_followees for source and destination nodes

In [30]:
%%time
if not os.path.isfile('saved_after_fea/storage_stage2.h5'):
    arr_train_cpy = np.array(df_train_cpy.iloc[:, :2])
    arr_test_cpy = np.array(df_test_cpy.iloc[:,:2])
    
    new_feat_train_cpy = np.array(list(map(lambda x: featurizationSet1(x[0], x[1]), arr_train_cpy)))
    new_feat_train_df = modpd.DataFrame(new_feat_train_cpy, columns = ['jaccard_followees','jaccard_followers','cosine_followees',
                                                                'cosine_followers', 'preferential_followees', 'preferential_followers'])
    df_train_cpy = modpd.concat([df_train_cpy, new_feat_train_df], axis=1)
    
    
    new_feat_test_cpy = np.array(list(map(lambda x: featurizationSet1(x[0], x[1]), arr_test_cpy)))
    new_feat_test_df = modpd.DataFrame(new_feat_test_cpy, columns = ['jaccard_followees','jaccard_followers','cosine_followees',
                                                                'cosine_followers', 'preferential_followees', 'preferential_followers'])
    df_test_cpy = modpd.concat([df_test_cpy, new_feat_test_df], axis=1)
    
    hdf = modpd.HDFStore('saved_after_fea/storage_stage2.h5')
    hdf.put('train_df',df_train_cpy, format='table', data_columns=True)
    hdf.put('test_df',df_test_cpy, format='table', data_columns=True)
    hdf.close()
else:
    df_train_cpy = modpd.read_hdf('saved_after_fea/storage_stage2.h5', 'train_df',mode='r')
    df_test_cpy = modpd.read_hdf('saved_after_fea/storage_stage2.h5', 'test_df',mode='r')

CPU times: user 11.6 s, sys: 14.1 s, total: 25.8 s
Wall time: 27.1 s


In [31]:
print(df_train_cpy.shape)
df_train_cpy.head()

(15100030, 15)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,jaccard_followees,jaccard_followers,cosine_followees,cosine_followers,preferential_followees,preferential_followers
0,273084,1505602,1,11,6,15,8,0,0,0.000000,0.000000,0.000000,0.000000,120.0,66.0
1,912810,1678443,1,10,8,10,8,1,1,0.058824,0.058824,0.111803,0.111803,80.0,80.0
2,365429,1523458,1,40,85,49,84,4,3,0.023077,0.033058,0.046761,0.068599,4116.0,3400.0
3,527014,1605979,1,0,1,1,0,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,1228116,471233,1,14,48,23,20,4,6,0.162162,0.068966,0.279751,0.154303,460.0,672.0


In [32]:
print(df_test_cpy.shape)
df_test_cpy.head()

(3775008, 15)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,jaccard_followees,jaccard_followers,cosine_followees,cosine_followers,preferential_followees,preferential_followers
0,848424,784690,1,6,14,6,9,1,0,0.000000,0.052632,0.000000,0.109109,54.0,84.0
1,1248963,444518,1,5,1,8,2,0,0,0.000000,0.000000,0.000000,0.000000,16.0,5.0
2,264224,132395,1,8,3,7,7,3,4,0.400000,0.375000,0.571429,0.612372,49.0,24.0
3,549680,326829,1,17,12,11,15,3,1,0.040000,0.115385,0.077850,0.210042,165.0,204.0
4,875380,1394902,1,21,29,20,25,8,7,0.184211,0.190476,0.313050,0.324176,500.0,609.0



## 5.2 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>adar index</li>
<li>is following back</li>
<li>belongs to same weakly connect components</li>
<li>shortest path between source and destination</li>
</ol>

In [33]:
%%time
if not os.path.isfile('saved_after_fea/storage_stage3.h5'):
    df_train_cpy['adar_index'] = df_train_cpy.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)
    df_test_cpy['adar_index'] = df_test_cpy.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)

    df_train_cpy['follows_back'] = df_train_cpy.apply(lambda row: follows_back(row['source_node'],row['destination_node']),axis=1)
    df_test_cpy['follows_back'] = df_test_cpy.apply(lambda row: follows_back(row['source_node'],row['destination_node']),axis=1)

    df_train_cpy['same_comp'] = df_train_cpy.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)
    df_test_cpy['same_comp'] = df_test_cpy.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)
    
    df_train_cpy['shortest_path'] = df_train_cpy.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)
    df_test_cpy['shortest_path'] = df_test_cpy.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)

    hdf = modpd.HDFStore('saved_after_fea/storage_stage3.h5')
    hdf.put('train_df',df_train_cpy, format='table', data_columns=True)
    hdf.put('test_df',df_test_cpy, format='table', data_columns=True)
    hdf.close()
else:
    df_train_cpy = modpd.read_hdf('saved_after_fea/storage_stage3.h5', 'train_df',mode='r')
    df_test_cpy = modpd.read_hdf('saved_after_fea/storage_stage3.h5', 'test_df',mode='r')

CPU times: user 16.3 s, sys: 19.4 s, total: 35.7 s
Wall time: 37.7 s


In [34]:
print(df_train_cpy.shape)
df_train_cpy.head()

(15100030, 19)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,jaccard_followees,jaccard_followers,cosine_followees,cosine_followers,preferential_followees,preferential_followers,adar_index,follows_back,same_comp,shortest_path
0,273084,1505602,1,11,6,15,8,0,0,0.000000,0.000000,0.000000,0.000000,120.0,66.0,0.000000,0,1,4
1,912810,1678443,1,10,8,10,8,1,1,0.058824,0.058824,0.111803,0.111803,80.0,80.0,1.183295,1,1,2
2,365429,1523458,1,40,85,49,84,4,3,0.023077,0.033058,0.046761,0.068599,4116.0,3400.0,2.073506,1,1,2
3,527014,1605979,1,0,1,1,0,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0,0,-1
4,1228116,471233,1,14,48,23,20,4,6,0.162162,0.068966,0.279751,0.154303,460.0,672.0,4.905057,0,1,2


In [35]:
print(df_test_cpy.shape)
df_test_cpy.head()

(3775008, 19)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,jaccard_followees,jaccard_followers,cosine_followees,cosine_followers,preferential_followees,preferential_followers,adar_index,follows_back,same_comp,shortest_path
0,848424,784690,1,6,14,6,9,1,0,0.000000,0.052632,0.000000,0.109109,54.0,84.0,0.000000,1,1,2
1,1248963,444518,1,5,1,8,2,0,0,0.000000,0.000000,0.000000,0.000000,16.0,5.0,0.000000,1,1,7
2,264224,132395,1,8,3,7,7,3,4,0.400000,0.375000,0.571429,0.612372,49.0,24.0,2.813472,1,1,2
3,549680,326829,1,17,12,11,15,3,1,0.040000,0.115385,0.077850,0.210042,165.0,204.0,1.047952,1,1,2
4,875380,1394902,1,21,29,20,25,8,7,0.184211,0.190476,0.313050,0.324176,500.0,609.0,6.147977,1,1,2


## 5.3 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>Weight Features
    <ul>
        <li>weight of incoming edges</li>
        <li>weight of outgoing edges</li>
        <li>weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges * weight of outgoing edges</li>
        <li>2*weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges + 2*weight of outgoing edges</li>
    </ul>
</li>
<li>Page Ranking of source</li>
<li>Page Ranking of dest</li>
<li>katz of source</li>
<li>katz of dest</li>
<li>hubs of source</li>
<li>hubs of dest</li>
<li>authorities_s of source</li>
<li>authorities_s of dest</li>
</ol>

#### Adding: Weight Features

In order to determine the similarity of nodes, an edge weight value was calculated between nodes. Edge weight decreases as the neighbor count goes up. Intuitively, consider one million people following a celebrity on a social network then chances are most of them never met each other or the celebrity. On the other hand, if a user has 30 contacts in his/her social network, the chances are higher that many of them know each other. 
`credit` - Graph-based Features for Supervised Link Prediction
William Cukierski, Benjamin Hamner, Bo Yang

\begin{equation}
W = \frac{1}{\sqrt{1+|X|}}
\end{equation}

it is directed graph so calculated Weighted in and Weighted out differently

In [41]:
#weight for source and destination of each link
Weight_in = {}
Weight_out = {}
for i in  tqdm(train_graph.nodes()):
    s1=set(train_graph.predecessors(i))
    w_in = 1.0/(np.sqrt(1+len(s1)))
    Weight_in[i]=w_in
    
    s2=set(train_graph.successors(i))
    w_out = 1.0/(np.sqrt(1+len(s2)))
    Weight_out[i]=w_out
    
#for imputing with mean, if node not avilable in dictionary return with mean walue
mean_weight_in = np.mean(list(Weight_in.values()))
mean_weight_out = np.mean(list(Weight_out.values()))

100%|████████████████████████████████████████████████████████████| 1780722/1780722 [00:15<00:00, 111890.83it/s]


In [42]:
%%time
if not os.path.isfile('saved_after_fea/storage_stage4.h5'):
    df_train_cpy['weight_in'] = df_train_cpy.destination_node.apply(lambda x: Weight_in.get(x,mean_weight_in))
    df_train_cpy['weight_out'] = df_train_cpy.source_node.apply(lambda x: Weight_out.get(x,mean_weight_out))

    df_test_cpy['weight_in'] = df_test_cpy.destination_node.apply(lambda x: Weight_in.get(x,mean_weight_in))
    df_test_cpy['weight_out'] = df_test_cpy.source_node.apply(lambda x: Weight_out.get(x,mean_weight_out))

    df_train_cpy['weight_f1'] = df_train_cpy.weight_in + df_train_cpy.weight_out
    df_train_cpy['weight_f2'] = df_train_cpy.weight_in * df_train_cpy.weight_out
    df_train_cpy['weight_f3'] = (2*df_train_cpy.weight_in + 1*df_train_cpy.weight_out)
    df_train_cpy['weight_f4'] = (1*df_train_cpy.weight_in + 2*df_train_cpy.weight_out)

    df_test_cpy['weight_f1'] = df_test_cpy.weight_in + df_test_cpy.weight_out
    df_test_cpy['weight_f2'] = df_test_cpy.weight_in * df_test_cpy.weight_out
    df_test_cpy['weight_f3'] = (2*df_test_cpy.weight_in + 1*df_test_cpy.weight_out)
    df_test_cpy['weight_f4'] = (1*df_test_cpy.weight_in + 2*df_test_cpy.weight_out)
    
    hdf = modpd.HDFStore('saved_after_fea/storage_stage4.h5')
    hdf.put('train_df',df_train_cpy, format='table', data_columns=True)
    hdf.put('test_df',df_test_cpy, format='table', data_columns=True)
    hdf.close()
else:
    df_train_cpy = modpd.read_hdf('saved_after_fea/storage_stage4.h5', 'train_df',mode='r')
    df_test_cpy = modpd.read_hdf('saved_after_fea/storage_stage4.h5', 'test_df',mode='r')

CPU times: user 7min 5s, sys: 59 s, total: 8min 4s
Wall time: 8min 30s


In [45]:
print(df_train_cpy.shape)
df_train_cpy.head()

(15100030, 25)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,jaccard_followees,...,adar_index,follows_back,same_comp,shortest_path,weight_in,weight_out,weight_f1,weight_f2,weight_f3,weight_f4
0,273084,1505602,1,11,6,15,8,0,0,0.000000,...,0.000000,0,1,4,0.377964,0.250000,0.627964,0.094491,1.005929,0.877964
1,912810,1678443,1,10,8,10,8,1,1,0.058824,...,1.183295,1,1,2,0.333333,0.301511,0.634845,0.100504,0.968178,0.936356
2,365429,1523458,1,40,85,49,84,4,3,0.023077,...,2.073506,1,1,2,0.107833,0.141421,0.249254,0.015250,0.357087,0.390675
3,527014,1605979,1,0,1,1,0,0,0,0.000000,...,0.000000,0,0,-1,0.707107,0.707107,1.414214,0.500000,2.121320,2.121320
4,1228116,471233,1,14,48,23,20,4,6,0.162162,...,4.905057,0,1,2,0.142857,0.204124,0.346981,0.029161,0.489838,0.551105


In [44]:
print(df_test_cpy.shape)
df_test_cpy.head()

(3775008, 25)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,jaccard_followees,...,adar_index,follows_back,same_comp,shortest_path,weight_in,weight_out,weight_f1,weight_f2,weight_f3,weight_f4
0,848424,784690,1,6,14,6,9,1,0,0.000000,...,0.000000,1,1,2,0.258199,0.377964,0.636163,0.097590,0.894362,1.014128
1,1248963,444518,1,5,1,8,2,0,0,0.000000,...,0.000000,1,1,7,0.707107,0.333333,1.040440,0.235702,1.747547,1.373773
2,264224,132395,1,8,3,7,7,3,4,0.400000,...,2.813472,1,1,2,0.500000,0.353553,0.853553,0.176777,1.353553,1.207107
3,549680,326829,1,17,12,11,15,3,1,0.040000,...,1.047952,1,1,2,0.277350,0.288675,0.566025,0.080064,0.843375,0.854700
4,875380,1394902,1,21,29,20,25,8,7,0.184211,...,6.147977,1,1,2,0.182574,0.218218,0.400792,0.039841,0.583366,0.619010


#### Adding : page ranking, katz centrality, hubs, authorities

In [47]:
#load pr, katz and hits if not loaded till this point
hits = pickle.load(open('saved_fea/hits.p', 'rb'))

katz = pickle.load(open('saved_fea/katz.p', 'rb'))
mean_katz = float(sum(katz.values())) / len(katz)

pr = pickle.load(open('saved_fea/page_rank.p', 'rb'))
mean_pr = np.mean(list(pr.values()))

In [90]:
%%time
if not os.path.isfile('saved_after_fea/storage_stage5.h5'):
    
    df_train_cpy['page_rank_s'] = df_train_cpy.source_node.apply(lambda x:pr.get(x,mean_pr))
    df_train_cpy['page_rank_d'] = df_train_cpy.destination_node.apply(lambda x:pr.get(x,mean_pr))

    df_test_cpy['page_rank_s'] = df_test_cpy.source_node.apply(lambda x:pr.get(x,mean_pr))
    df_test_cpy['page_rank_d'] = df_test_cpy.destination_node.apply(lambda x:pr.get(x,mean_pr))

    
    df_train_cpy['katz_s'] = df_train_cpy.source_node.apply(lambda x: katz.get(x,mean_katz))
    df_train_cpy['katz_d'] = df_train_cpy.destination_node.apply(lambda x: katz.get(x,mean_katz))

    df_test_cpy['katz_s'] = df_test_cpy.source_node.apply(lambda x: katz.get(x,mean_katz))
    df_test_cpy['katz_d'] = df_test_cpy.destination_node.apply(lambda x: katz.get(x,mean_katz))

    
    df_train_cpy['hubs_s'] = df_train_cpy.source_node.apply(lambda x: hits[0].get(x,0))
    df_train_cpy['hubs_d'] = df_train_cpy.destination_node.apply(lambda x: hits[0].get(x,0))

    df_test_cpy['hubs_s'] = df_test_cpy.source_node.apply(lambda x: hits[0].get(x,0))
    df_test_cpy['hubs_d'] = df_test_cpy.destination_node.apply(lambda x: hits[0].get(x,0))
    
    
    df_train_cpy['authorities_s'] = df_train_cpy.source_node.apply(lambda x: hits[1].get(x,0))
    df_train_cpy['authorities_d'] = df_train_cpy.destination_node.apply(lambda x: hits[1].get(x,0))

    df_test_cpy['authorities_s'] = df_test_cpy.source_node.apply(lambda x: hits[1].get(x,0))
    df_test_cpy['authorities_d'] = df_test_cpy.destination_node.apply(lambda x: hits[1].get(x,0))

    hdf = modpd.HDFStore('saved_after_fea/storage_stage5.h5')
    hdf.put('train_df',df_train_cpy, format='table', data_columns=True)
    hdf.put('test_df',df_test_cpy, format='table', data_columns=True)
    hdf.close()
else:
    df_train_cpy = modpd.read_hdf('saved_after_fea/storage_stage5.h5', 'train_df',mode='r')
    df_test_cpy = modpd.read_hdf('saved_after_fea/storage_stage5.h5', 'test_df',mode='r')

CPU times: user 30.1 s, sys: 26.9 s, total: 57 s
Wall time: 52.6 s


In [91]:
print(df_train_cpy.shape)
df_train_cpy.head()

(15100030, 33)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,jaccard_followees,...,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d
0,273084,1505602,1,11,6,15,8,0,0,0.000000,...,1.005929,0.877964,2.045290e-06,3.459963e-07,0.000773,0.000756,1.943132e-13,1.941103e-13,9.226339e-16,2.231877e-15
1,912810,1678443,1,10,8,10,8,1,1,0.058824,...,0.968178,0.936356,1.039181e-06,1.793806e-06,0.000771,0.000761,6.223983e-16,7.761651e-18,9.501394e-16,5.142398e-18
2,365429,1523458,1,40,85,49,84,4,3,0.023077,...,0.357087,0.390675,1.033022e-06,3.096856e-06,0.000911,0.001099,2.212482e-13,2.563941e-13,7.761986e-14,6.291902e-14
3,527014,1605979,1,0,1,1,0,0,0,0.000000,...,2.121320,2.121320,1.655650e-07,6.428994e-07,0.000731,0.000735,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,1228116,471233,1,14,48,23,20,4,6,0.162162,...,0.489838,0.551105,8.348032e-07,2.665876e-06,0.000793,0.000922,6.917913e-17,1.320470e-16,5.963786e-18,3.407897e-17


In [92]:
print(df_test_cpy.shape)
df_test_cpy.head()

(3775008, 33)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,jaccard_followees,...,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d
0,848424,784690,1,6,14,6,9,1,0,0.000000,...,0.894362,1.014128,6.557971e-07,1.559547e-06,0.000754,0.000786,3.243237e-16,1.745627e-16,2.969838e-15,9.269213e-14
1,1248963,444518,1,5,1,8,2,0,0,0.000000,...,1.747547,1.373773,1.115895e-06,2.451207e-07,0.000750,0.000735,1.963513e-17,1.924496e-20,1.727944e-18,4.482147e-20
2,264224,132395,1,8,3,7,7,3,4,0.400000,...,1.353553,1.207107,5.176389e-07,3.594821e-07,0.000763,0.000743,3.532699e-14,3.543278e-14,8.377534e-15,4.484954e-15
3,549680,326829,1,17,12,11,15,3,1,0.040000,...,0.843375,0.854700,8.354002e-07,7.644794e-07,0.000800,0.000779,5.609577e-13,2.220721e-13,2.822190e-11,2.426166e-14
4,875380,1394902,1,21,29,20,25,8,7,0.184211,...,0.583366,0.619010,1.500364e-06,1.550158e-06,0.000813,0.000846,2.408301e-17,1.167624e-16,1.431057e-16,1.911917e-16


## 5.4 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>SVD features for both source and destination</li>
<li>SVD dot product of source and destination svd vectors</li> 
</ol>

#### Adding : SVD Features, SVD Dot

In [39]:
#sadj_col -> [1,3,4,5,8,11,24,26,28]
#sadj_dict -> [1:0, 3:1, 4:2, 5:3, 8:4,..28:8]

sadj_col = sorted(train_graph.nodes())
sadj_dict = { val:idx for idx,val in enumerate(sadj_col)}

In [40]:
def svd(x, S):
    try:
        #get the index of user node and use the index to get the vector at that index from U or V
        z = sadj_dict[x]
        return S[z]
    #when creating feature vector for test, if user not present in train graph then return 0 vector
    except:
        return [0,0,0,0,0,0]

In [41]:
Adj = nx.adjacency_matrix(train_graph,nodelist=sorted(train_graph.nodes())).asfptype()

In [42]:
U, s, V = svds(Adj, k = 6)
print('Adjacency matrix Shape',Adj.shape)
print('U Shape',U.shape)
print('V Shape',V.shape)
print('s Shape',s.shape)

Adjacency matrix Shape (1780722, 1780722)
U Shape (1780722, 6)
V Shape (6, 1780722)
s Shape (6,)


In [66]:
def svd_dot(ai1, ai2, ai3, ai4, ai5, ai6, aj1, aj2, aj3, aj4, aj5, aj6):
    a_i = np.array([ai1, ai2, ai3, ai4, ai5, ai6])
    a_j = np.array([aj1, aj2, aj3, aj4, aj5, aj6])
    return np.dot(a_i, a_j)

In [93]:
%%time
if not os.path.isfile('saved_after_fea/storage_stage6.h5'):
    arr_train_cpy = df_train_cpy.iloc[:, :2].values
    arr_test_cpy = df_test_cpy.iloc[:, :2].values

    train_svd_s_u = np.array(list(map(lambda x: svd(x[0],U), arr_train_cpy)))
    train_svd_d_u = np.array(list(map(lambda x: svd(x[1],U), arr_train_cpy)))
    train_svd_u = np.hstack((train_svd_s_u,train_svd_d_u))
    train_svd_dot_u = np.array(list(map(lambda x: svd_dot(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11]), train_svd_u))).reshape(-1,1)

    train_svd_s_v = np.array(list(map(lambda x: svd(x[0],V.T), arr_train_cpy)))
    train_svd_d_v = np.array(list(map(lambda x: svd(x[1],V.T), arr_train_cpy)))
    train_svd_v = np.hstack((train_svd_s_v, train_svd_d_v))
    train_svd_dot_v = np.array(list(map(lambda x: svd_dot(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11]), train_svd_v))).reshape(-1,1)
    
    train_svd_arr = np.concatenate((train_svd_u,train_svd_v,train_svd_dot_u,train_svd_dot_v), axis = 1)
    train_svd_df = modpd.DataFrame(train_svd_arr, columns =['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6',
                                                       'svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6',
                                                       'svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',
                                                       'svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6',
                                                        'svd_dot_u','svd_dot_v'])
    df_train_cpy = modpd.concat([df_train_cpy, train_svd_df], axis=1)
    
    
    
    test_svd_s_u = np.array(list(map(lambda x: svd(x[0],U), arr_test_cpy)))
    test_svd_d_u = np.array(list(map(lambda x: svd(x[1],U), arr_test_cpy)))
    test_svd_u = np.hstack((test_svd_s_u,test_svd_d_u))
    test_svd_dot_u = np.array(list(map(lambda x: svd_dot(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11]), test_svd_u))).reshape(-1,1)

    test_svd_s_v = np.array(list(map(lambda x: svd(x[0],V.T), arr_test_cpy)))
    test_svd_d_v = np.array(list(map(lambda x: svd(x[1],V.T), arr_test_cpy)))
    test_svd_v = np.hstack((test_svd_s_v,test_svd_d_v))
    test_svd_dot_v = np.array(list(map(lambda x: svd_dot(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11]), test_svd_v))).reshape(-1,1)
    
    test_svd_arr = np.concatenate((test_svd_u,test_svd_v,test_svd_dot_u,test_svd_dot_v), axis = 1)
    test_svd_df = modpd.DataFrame(test_svd_arr, columns =['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6',
                                                       'svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6',
                                                       'svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',
                                                       'svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6',
                                                        'svd_dot_u','svd_dot_v'])
    df_test_cpy = modpd.concat([df_test_cpy, test_svd_df], axis=1)

    hdf = modpd.HDFStore('saved_after_fea/storage_stage6.h5')
    hdf.put('train_df',df_train_cpy, format='table', data_columns=True)
    hdf.put('test_df',df_test_cpy, format='table', data_columns=True)
    hdf.close()

else:
    df_train_cpy = modpd.read_hdf('saved_after_fea/storage_stage6.h5', 'train_df',mode='r')
    df_test_cpy = modpd.read_hdf('saved_after_fea/storage_stage6.h5', 'test_df',mode='r')

CPU times: user 25min 59s, sys: 2min 58s, total: 28min 57s
Wall time: 29min 14s


In [94]:
print(df_train_cpy.shape)
df_train_cpy.head()

(15100030, 59)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,jaccard_followees,...,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,svd_dot_u,svd_dot_v
0,273084,1505602,1,11,6,15,8,0,0,0.000000,...,8.108438e-13,1.719704e-14,-1.355369e-12,4.675320e-13,1.128586e-06,6.616715e-14,9.771079e-13,4.160011e-14,1.114951e-11,2.238777e-12
1,912810,1678443,1,10,8,10,8,1,1,0.058824,...,2.595389e-10,1.770966e-14,-2.786380e-13,6.863195e-15,3.891589e-11,1.423121e-14,1.053573e-13,9.584523e-17,8.999196e-20,1.183940e-19
2,365429,1523458,1,40,85,49,84,4,3,0.023077,...,6.554821e-10,1.446762e-12,-9.031624e-10,2.911246e-11,4.190039e-05,8.469404e-12,3.504144e-11,1.172753e-12,3.514353e-08,2.890101e-09
3,527014,1605979,1,0,1,1,0,0,0,0.000000,...,0.000000e+00,0.000000e+00,3.477907e-22,-3.040018e-23,5.749145e-23,1.687944e-22,1.629802e-21,9.523347e-22,1.648122e-38,0.000000e+00
4,1228116,471233,1,14,48,23,20,4,6,0.162162,...,3.750496e-11,1.111372e-16,-1.031682e-14,1.736398e-07,2.623472e-10,2.144779e-14,1.503898e-09,6.348404e-16,2.674128e-15,1.405864e-15


In [95]:
print(df_test_cpy.shape)
df_test_cpy.head()

(3775008, 59)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,jaccard_followees,...,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,svd_dot_u,svd_dot_v
0,848424,784690,1,6,14,6,9,1,0,0.000000,...,4.341620e-13,5.535504e-14,-9.994077e-10,5.791914e-10,3.512351e-07,2.486660e-09,2.771146e-09,1.727695e-12,8.425175e-20,2.074802e-17
1,1248963,444518,1,5,1,8,2,0,0,0.000000,...,2.906688e-15,3.220585e-17,-1.076246e-18,2.511389e-15,8.853315e-17,1.366578e-16,7.887724e-17,8.361642e-19,1.974053e-26,1.160816e-28
2,264224,132395,1,8,3,7,7,3,4,0.400000,...,1.670538e-12,1.561495e-13,-1.682506e-12,2.274655e-13,8.221382e-11,4.182890e-13,1.365162e-12,8.359540e-14,2.482757e-21,9.952720e-21
3,549680,326829,1,17,12,11,15,3,1,0.040000,...,9.557320e-11,5.260301e-10,-6.671577e-12,4.936914e-12,2.297719e-09,1.314355e-11,1.869695e-11,4.522149e-13,2.182195e-16,3.103432e-18
4,875380,1394902,1,21,29,20,25,8,7,0.184211,...,6.445215e-12,2.667349e-15,-3.860894e-13,7.424223e-12,1.228955e-11,3.050131e-12,6.923486e-12,3.563640e-15,1.708571e-22,3.671139e-22
